In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import datetime
import logging
import os

In [9]:
# 获取脚本的当前路径，避免计划执行时路径出错
home_dir = os.path.dirname(os.path.realpath(__file__))
# 更换workding directory
working_directory = home_dir
os.chdir(working_directory)

NameError: name '__file__' is not defined

In [82]:
# pip install selenium

In [83]:
# Date
today_date = datetime.date.today() + datetime.timedelta(days=0)
yesterday_date = datetime.date.today() + datetime.timedelta(days=-1)
seven_days_before_date = datetime.date.today() + datetime.timedelta(days=-7)

today_date_string = today_date.strftime('%Y_%m_%d')

In [84]:
# os.getcwd()

In [85]:
log_file_name = 'shopee_add_and_cancel_fans_log\\shopee_add_and_cancel_fans_log_' + today_date_string + '.txt'

# logging.basicConfig(filename=log_file_name, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [86]:
# 屏幕最大化，且指定下载目录
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

prefs = {"profile.default_content_settings.popups": 0,
         "directory_upgrade": True}
options.add_experimental_option("prefs", prefs)

In [87]:
# 使用chromedriver才可以用开发者权限
chrome_driver_path = ".//chrome_driver//chromedriver.exe"
# browser = webdriver.Chrome(chrome_driver_path, chrome_options=options)

In [88]:
# 所需参数
# 站点；站点后缀；账户；密码；站点top卖家shopid；top卖家username（方便识别）
# shop_list = [['tw', 'tw', 23070969, 'poweradapter.tw', 'kuangyiqiao1991', 9469128, 'alonso.tw'],
#              ['tw', 'tw', 25482220, 'tengus.tw', 'tengus1803', 23068230, 'iu.tw'],
#              ['tw', 'tw', 58707738, 'qianjiaozi', 'qianjiaozi888', 26513957, 'beibei.tw'],
#              ['tw', 'tw', 66377809, 'qianjiaozi1', 'qianjiaozi888', 23068230, 'iu.tw'],
#              ['tw', 'tw', 62416366, 'tengus1.tw', 'tengus1803', 23068230, 'iu.tw'],
#              ['tw', 'tw', 62416544, 'tengus2.tw', 'tengus1803', 23068230, 'iu.tw'],
#              ['tw', 'tw', 63534861, 'qianjiaozitw1', 'tengus1803', 23068230, 'iu.tw'],
#              ['tw', 'tw', 62887142, 'yilanlu.tw', 'yilanlu888', 23481977, 'huoyi.tw'],
#              ['my', 'com.my', 53580963, 'qianjiaozi.my', 'tengus1803', 10891137, 'winners.my'],
#              ['my', 'com.my', 59848325, 'tengus.my', 'tengus1803', 10891137, 'winners.my'],
#              ['my', 'com.my', 62418141, 'tengus1.my', 'tengus1803', 10891137, 'winners.my'],
#              ['my', 'com.my', 62418493, 'tengus2.my', 'tengus1803', 10891137, 'winners.my'],
#              ['id', 'co.id', 59846508, 'tengus.id', 'tengus1803', 28135012, 'sunnyfun.id'],
#              ['id', 'co.id', 62417386, 'tengus1.id', 'tengus1803', 28135012, 'sunnyfun.id'],
#              ['id', 'co.id', 62417551, 'tengus2.id', 'tengus1803', 28135012, 'sunnyfun.id']]

shop_list = [['tw', 'tw', 23070969, 'poweradapter.tw', 'kuangyiqiao1991', 9469128, 'alonso.tw'],
             ['th', 'co.th', 117213614, 'tengus.th', 'tengus1803', 25926687, 'xiaozhainv']]

In [89]:
# 转换为dataframe
shop_df_columns = ['site', 'site_suffix', 'shopid', 'acc', 'pwd', 'top_shop_id', 'top_shop_username']
shop_df = pd.DataFrame(shop_list, columns=shop_df_columns)

In [90]:
# 关注及关注中多语言
following_language = ['关注中', '關注中', 'Following', 'Mengikuti', 'กำลังติดตาม']
not_following_language = ['+ 关注', '+ 關注', '+ Follow', '+ Ikuti', "+ ติดตาม"]

In [91]:
# 取关后关注函数
def add_and_cancel_fans(site, site_suffix, shopid, acc, pwd, top_shop_id, top_shop_username):
    # 打开浏览器
    logging.info('Open the browser.')
    browser = webdriver.Chrome(chrome_driver_path, chrome_options=options)
    
    # main page
    main_page_url = "http://shopee." + site_suffix
    browser.get(main_page_url)
    time.sleep(10)
    
    # remove ads, refresh again
    browser.refresh()
    time.sleep(10)
    browser.refresh()
    time.sleep(10)

    # if the shop is on my site, should press the button to choose language
    if site == 'my':
        language_selector = browser.find_elements_by_css_selector('.shopee-button-outline.shopee-button-outline--primary-reverse')
        language_selector[2].click()
        logging.info('Choose Chinese as language.')
        time.sleep(10)
    
    # login
    logging.info('Start to login.')
    LoginElem = browser.find_elements_by_css_selector('.navbar__link.navbar__link--account.navbar__link--tappable.navbar__link--hoverable.navbar__link-text.navbar__link-text--medium')
    
    # 点击登入，弹出账户密码输入框
    LoginElem[1].click()
    time.sleep(10)
    
    # 输入
    acc_password_input_elem = browser.find_elements_by_css_selector('._2QBp41._1b-IZR')
    try:
        acc_password_input_elem[0].click()
        acc_password_input_elem[0].send_keys(acc)
        acc_password_input_elem[1].click()
        acc_password_input_elem[1].send_keys(pwd)
    except:
        time.sleep(30)
        acc_password_input_elem = browser.find_elements_by_css_selector('._2QBp41._1b-IZR')
        acc_password_input_elem[0].click()
        acc_password_input_elem[0].send_keys(acc)
        acc_password_input_elem[1].click()
        acc_password_input_elem[1].send_keys(pwd)
    time.sleep(10)
    
    # 点击提交
    Login_button_elem = browser.find_elements_by_css_selector('._2DvX7K._3j9-lD._3ddytl.SjORHu')
    Login_button_elem[0].click()
    logging.info('Login completed.')
    time.sleep(10)
    
    # switch to my following list
    logging.info('Switch to my following list.')
    my_fans_list_page = 'https://shopee.' + site_suffix + '/shop/' + str(shopid) + '/following/?__classic__=1'
    browser.get(my_fans_list_page)
    
    # cancel certain number of following fans
    to_cancel_num_of_following = 400
    num_of_following_display = len(browser.find_elements_by_css_selector('.clickable_area.middle-centered-div'))
    
    # in case the following number is not greater than 500
    max_roll_time = 100
    roll_time = 0
    
    # page down until we get at least 500 fans to cancel
    while num_of_following_display <= 500 and roll_time <= max_roll_time:
        body = browser.find_element_by_css_selector('body')
        body.send_keys(Keys.END)
        time.sleep(2)
        num_of_following_display = len(browser.find_elements_by_css_selector('.clickable_area.middle-centered-div'))
        logging.info(str(num_of_following_display) + ' following on the list.')
        roll_time += 1
        logging.info('Roll the page for ' + str(roll_time) + ' time.')
    following_buttons = browser.find_elements_by_css_selector('.btn-follow.active.follow.L14')
    
    # actual following to cancel
    to_cancel_num_of_following = min(num_of_following_display, to_cancel_num_of_following)
    
    # page up to the top
    scroll_to_the_top = browser.find_element_by_css_selector('body').send_keys(Keys.CONTROL + Keys.HOME)
    
    # start to cancel following
    logging.info('Start to cancel following.')
    i = 0
    total_cancel_num_of_following = to_cancel_num_of_following
    while i <= total_cancel_num_of_following - 1:
        while to_cancel_num_of_following > 0:
            logging.info('Now run the ' + str(i + 1) + ' time.')
            if following_buttons[i].text in following_language:
                buyer_shopid = following_buttons[i].get_attribute('shopid')
                following_buttons[i].click()
                time.sleep(1)
                to_cancel_num_of_following -= 1
                i += 1
                logging.info(acc + ': ' + str(buyer_shopid) + ' is not following now, ' + str(to_cancel_num_of_following) + ' following remains.')
    
    # add fans from Top Seller's followers
    logging.info('Switch to top seller follower page.')
    top_shop_url = 'https://shopee.' + site_suffix + '/shop/' + str(top_shop_id) + '/followers/?__classic__=1'
    browser.get(top_shop_url)
    
    # add certain number of fans
    to_add_num_of_following = 400
    num_of_following_display = len(browser.find_elements_by_css_selector('.clickable_area.middle-centered-div'))
    
    # page down until we get at least 400 fans to cancel
    logging.info('Start to roll down.')
    while num_of_following_display <= 1000:
        body = browser.find_element_by_css_selector('body')
        # body.send_keys(Keys.PAGE_DOWN)
        body.send_keys(Keys.END)
        time.sleep(2)
        num_of_following_display = len(browser.find_elements_by_css_selector('.clickable_area.middle-centered-div'))
    following_buttons = browser.find_elements_by_css_selector('.btn-follow.follow.L14')
    
    # start to follow buyers from top shop
    logging.info('Start to follow buyers.')
    i = 0
    total_add_num_of_following = to_add_num_of_following
    scroll_to_the_top = browser.find_element_by_css_selector('body').send_keys(Keys.CONTROL + Keys.HOME)

    while i <= total_add_num_of_following - 1:  
        while to_add_num_of_following > 0:
            logging.info('Now run the ' + str(i + 1) + ' time.')
            if following_buttons[i].text in not_following_language:
                # print(following_buttons[i].text)
                buyer_shopid = following_buttons[i].get_attribute('shopid')
                following_buttons[i].click()
                time.sleep(1)
                to_add_num_of_following -= 1
                logging.info(acc + ': ' + str(buyer_shopid) + ' is following now, ' + str(to_add_num_of_following) + ' following remains.')
            else:
                logging.info('Skip this one. It\'s following already.')
            i += 1    
    
    # 关闭
    browser.quit()

In [92]:
# 历遍所有shop
for index, my_shop in shop_df.iterrows():
    site = my_shop[0]
    site_suffix = my_shop[1]
    shopid = my_shop[2]
    acc = my_shop[3]
    pwd = my_shop[4]
    top_shop_id = my_shop[5]
    top_shop_username = my_shop[6]
    
    # 执行
    try:
        logging.info('Now it\'s running the script on ' + acc + '.')
        add_and_cancel_fans(site, site_suffix, shopid, acc, pwd, top_shop_id, top_shop_username)
        logging.info('Finished running the script on ' + acc + '.')
    except Exception as err:
        logging.info('An exception occurred: ' + str(err) + '.')

2019-09-07 17:16:32,663 - INFO - Now it's running the script on tengus.th.
2019-09-07 17:16:32,663 - INFO - Open the browser.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """
2019-09-07 17:17:36,710 - INFO - Start to login.
2019-09-07 17:17:57,682 - INFO - Login completed.
2019-09-07 17:18:07,685 - INFO - Switch to top seller follower page.
2019-09-07 17:18:50,080 - INFO - Start to roll down.
2019-09-07 17:21:47,092 - INFO - Start to follow buyers.
2019-09-07 17:21:47,160 - INFO - Now run the 1 time.
2019-09-07 17:21:48,335 - INFO - tengus.th: 7855720 is following now, 399 following remains.
2019-09-07 17:21:48,335 - INFO - Now run the 2 time.
2019-09-07 17:21:49,475 - INFO - tengus.th: 985503 is following now, 398 following remains.
2019-09-07 17:21:49,477 - INFO - Now run the 3 time.
2019-09-07 17:21:50,595 - INFO - tengus.th: 92577622 is following now, 397 following remains.
2019-09-07 17:21:50,596 - 

2019-09-07 17:22:45,319 - INFO - Now run the 53 time.
2019-09-07 17:22:46,507 - INFO - tengus.th: 17490364 is following now, 347 following remains.
2019-09-07 17:22:46,507 - INFO - Now run the 54 time.
2019-09-07 17:22:47,632 - INFO - tengus.th: 48386672 is following now, 346 following remains.
2019-09-07 17:22:47,632 - INFO - Now run the 55 time.
2019-09-07 17:22:48,741 - INFO - tengus.th: 9763224 is following now, 345 following remains.
2019-09-07 17:22:48,741 - INFO - Now run the 56 time.
2019-09-07 17:22:49,835 - INFO - tengus.th: 89124094 is following now, 344 following remains.
2019-09-07 17:22:49,835 - INFO - Now run the 57 time.
2019-09-07 17:22:50,944 - INFO - tengus.th: 101620162 is following now, 343 following remains.
2019-09-07 17:22:50,944 - INFO - Now run the 58 time.
2019-09-07 17:22:52,056 - INFO - tengus.th: 19932842 is following now, 342 following remains.
2019-09-07 17:22:52,057 - INFO - Now run the 59 time.
2019-09-07 17:22:53,190 - INFO - tengus.th: 4120736 is fol

2019-09-07 17:23:49,182 - INFO - tengus.th: 169768088 is following now, 292 following remains.
2019-09-07 17:23:49,183 - INFO - Now run the 109 time.
2019-09-07 17:23:50,304 - INFO - tengus.th: 27771858 is following now, 291 following remains.
2019-09-07 17:23:50,304 - INFO - Now run the 110 time.
2019-09-07 17:23:51,413 - INFO - tengus.th: 40119678 is following now, 290 following remains.
2019-09-07 17:23:51,413 - INFO - Now run the 111 time.
2019-09-07 17:23:52,538 - INFO - tengus.th: 22618130 is following now, 289 following remains.
2019-09-07 17:23:52,538 - INFO - Now run the 112 time.
2019-09-07 17:23:53,647 - INFO - tengus.th: 33277193 is following now, 288 following remains.
2019-09-07 17:23:53,647 - INFO - Now run the 113 time.
2019-09-07 17:23:54,788 - INFO - tengus.th: 62480059 is following now, 287 following remains.
2019-09-07 17:23:54,788 - INFO - Now run the 114 time.
2019-09-07 17:23:55,913 - INFO - tengus.th: 176693063 is following now, 286 following remains.
2019-09-07

2019-09-07 17:24:50,507 - INFO - Now run the 164 time.
2019-09-07 17:24:51,647 - INFO - tengus.th: 135110658 is following now, 236 following remains.
2019-09-07 17:24:51,647 - INFO - Now run the 165 time.
2019-09-07 17:24:52,741 - INFO - tengus.th: 22970147 is following now, 235 following remains.
2019-09-07 17:24:52,741 - INFO - Now run the 166 time.
2019-09-07 17:24:53,835 - INFO - tengus.th: 2707702 is following now, 234 following remains.
2019-09-07 17:24:53,835 - INFO - Now run the 167 time.
2019-09-07 17:24:54,944 - INFO - tengus.th: 144654437 is following now, 233 following remains.
2019-09-07 17:24:54,944 - INFO - Now run the 168 time.
2019-09-07 17:24:56,069 - INFO - tengus.th: 94221447 is following now, 232 following remains.
2019-09-07 17:24:56,069 - INFO - Now run the 169 time.
2019-09-07 17:24:57,185 - INFO - tengus.th: 163672389 is following now, 231 following remains.
2019-09-07 17:24:57,186 - INFO - Now run the 170 time.
2019-09-07 17:24:58,288 - INFO - tengus.th: 79994

2019-09-07 17:25:51,866 - INFO - Now run the 219 time.
2019-09-07 17:25:52,960 - INFO - tengus.th: 84499840 is following now, 181 following remains.
2019-09-07 17:25:52,960 - INFO - Now run the 220 time.
2019-09-07 17:25:54,069 - INFO - tengus.th: 6266891 is following now, 180 following remains.
2019-09-07 17:25:54,070 - INFO - Now run the 221 time.
2019-09-07 17:25:55,183 - INFO - tengus.th: 14022972 is following now, 179 following remains.
2019-09-07 17:25:55,184 - INFO - Now run the 222 time.
2019-09-07 17:25:56,309 - INFO - tengus.th: 110803633 is following now, 178 following remains.
2019-09-07 17:25:56,310 - INFO - Now run the 223 time.
2019-09-07 17:25:57,397 - INFO - tengus.th: 173856412 is following now, 177 following remains.
2019-09-07 17:25:57,397 - INFO - Now run the 224 time.
2019-09-07 17:25:58,507 - INFO - tengus.th: 12860390 is following now, 176 following remains.
2019-09-07 17:25:58,507 - INFO - Now run the 225 time.
2019-09-07 17:25:59,616 - INFO - tengus.th: 354874

2019-09-07 17:26:54,310 - INFO - Now run the 274 time.
2019-09-07 17:26:55,397 - INFO - tengus.th: 110230509 is following now, 126 following remains.
2019-09-07 17:26:55,397 - INFO - Now run the 275 time.
2019-09-07 17:26:56,491 - INFO - tengus.th: 173335580 is following now, 125 following remains.
2019-09-07 17:26:56,491 - INFO - Now run the 276 time.
2019-09-07 17:26:57,600 - INFO - tengus.th: 169021410 is following now, 124 following remains.
2019-09-07 17:26:57,600 - INFO - Now run the 277 time.
2019-09-07 17:26:58,694 - INFO - tengus.th: 21412411 is following now, 123 following remains.
2019-09-07 17:26:58,694 - INFO - Now run the 278 time.
2019-09-07 17:26:59,807 - INFO - tengus.th: 3601642 is following now, 122 following remains.
2019-09-07 17:26:59,808 - INFO - Now run the 279 time.
2019-09-07 17:27:00,897 - INFO - tengus.th: 26042847 is following now, 121 following remains.
2019-09-07 17:27:00,897 - INFO - Now run the 280 time.
2019-09-07 17:27:02,038 - INFO - tengus.th: 13318

2019-09-07 17:27:56,725 - INFO - tengus.th: 174816856 is following now, 71 following remains.
2019-09-07 17:27:56,725 - INFO - Now run the 330 time.
2019-09-07 17:27:57,850 - INFO - tengus.th: 154872244 is following now, 70 following remains.
2019-09-07 17:27:57,850 - INFO - Now run the 331 time.
2019-09-07 17:27:58,944 - INFO - tengus.th: 19265847 is following now, 69 following remains.
2019-09-07 17:27:58,944 - INFO - Now run the 332 time.
2019-09-07 17:28:00,069 - INFO - tengus.th: 6779669 is following now, 68 following remains.
2019-09-07 17:28:00,069 - INFO - Now run the 333 time.
2019-09-07 17:28:01,157 - INFO - tengus.th: 175248616 is following now, 67 following remains.
2019-09-07 17:28:01,158 - INFO - Now run the 334 time.
2019-09-07 17:28:02,281 - INFO - tengus.th: 1159324 is following now, 66 following remains.
2019-09-07 17:28:02,282 - INFO - Now run the 335 time.
2019-09-07 17:28:03,366 - INFO - tengus.th: 77892859 is following now, 65 following remains.
2019-09-07 17:28:0

2019-09-07 17:28:58,183 - INFO - Now run the 385 time.
2019-09-07 17:28:59,277 - INFO - tengus.th: 9332693 is following now, 15 following remains.
2019-09-07 17:28:59,278 - INFO - Now run the 386 time.
2019-09-07 17:29:00,382 - INFO - tengus.th: 28585246 is following now, 14 following remains.
2019-09-07 17:29:00,382 - INFO - Now run the 387 time.
2019-09-07 17:29:01,491 - INFO - tengus.th: 55490186 is following now, 13 following remains.
2019-09-07 17:29:01,492 - INFO - Now run the 388 time.
2019-09-07 17:29:02,600 - INFO - tengus.th: 26305149 is following now, 12 following remains.
2019-09-07 17:29:02,600 - INFO - Now run the 389 time.
2019-09-07 17:29:03,694 - INFO - tengus.th: 176429616 is following now, 11 following remains.
2019-09-07 17:29:03,694 - INFO - Now run the 390 time.
2019-09-07 17:29:04,819 - INFO - tengus.th: 131934176 is following now, 10 following remains.
2019-09-07 17:29:04,819 - INFO - Now run the 391 time.
2019-09-07 17:29:05,932 - INFO - tengus.th: 40069329 is 

In [93]:
# browser = webdriver.Chrome(chrome_driver_path, chrome_options=options)

In [94]:
# browser.get('https://shopee.com.my')

In [95]:
# language_selector = browser.find_elements_by_css_selector('.shopee-button-outline.shopee-button-outline--primary-reverse')
# len(language_selector)

In [96]:
#language_selector[2].click()